In [75]:
# Autor: Daniel Pinto
# DP & greediness
# Fecha: 2021/10/14 YYYY/MM/DD
from typing import List, TypeVar, Tuple, Any, Callable, Optional, Generic, Dict
from hypothesis import given, strategies as st
from IPython.display import Markdown, display
from dataclasses import dataclass, field
from __future__ import annotations 
from copy import deepcopy
from collections.abc import  Iterable
import graphviz as gv
from abc import ABCMeta, abstractmethod
from math import inf

def display_(s : str) -> None:
    '''
    A way to display strings with markdown 
    in jupyter.
    '''
    display(
        Markdown(s)
    )


SUCCESS_COLOR = '#4BB543'
ERROR_COLOR   = '#B00020'

def color_text(s : str, color : str =SUCCESS_COLOR ) -> str:
    return f"<span style='color:{color}'> {s} </span>."


a      = TypeVar('a')
b      = TypeVar('b')
c      = TypeVar('c')
T      = TypeVar('T')

# Selection Sort

Dado un arreglo `arr`, selection sort crea un arreglo ordenado `ord` de la siguiente forma:

* `ord[0]   = min(arr)`
* `ord[i+1] = min(arr // ord[0:i])` 

Es decir, en cada iteracion inserta el proximo elemento minimo.

$$
\sum_{i=0}^n (n-i) \sim n^2
$$


In [76]:
# [0,1,2,5,9,20]
# \theta n^2
# O(n^2) -> O(n) 
# [5,4,3,2]
# [1,2]
# s
# Version in place:
def Selection_sort(xs : List[a]):
    for i in range(len(xs)):
        min_so_far = i
        for j in range(i,len(xs)):
            if xs[j] < xs[min_so_far]:
                min_so_far = j
        xs[i], xs[min_so_far] = xs[min_so_far], xs[i]

xs = [5,3,9,0,1,5,2,1,3,7]
Selection_sort(xs)
print(xs)


[0, 1, 1, 2, 3, 3, 5, 5, 7, 9]


# Quicksort

Sea $a,b$, $a \leq b$ dos posiciones cualquiera del arreglo `arr`, quicksort sortea el subarreglo `arr[a:b+1]` de la siguiente manera:

1. Si `arr` es de longitud 1 o 0, esta ordenado por definicion
2. Si `arr` es de longitud 2, se ordena de forma trivial
3. Si `arr` es de longitud 3 o mas, entonces dejemos que $a \leq i \leq b$ sea un indice cualquiera. Particionaremos el arreglo en dos mitades: `arr[a:i]` tendra todos los elementos que sean **menores o iguales**  a `arr[i]`, y `arr[i:b+1]` tendra todos los elementos que sean **mayores** a `arr[i]`.
4. Ordenamos ambas mitades usando quicksort

In [78]:
# n^2
# n*log(n)
# n 
# [5,3,9,0,1,5,2,1,3,7]
# pivot = 5
# l = [5,3,0,1,2,1,3] -> [0,1,1,2,3,3,5] 
# r = [9,7]           -> [7,9]

# l [] 
# r resto de los elementos
# particion
# [0,1,1,2,3,3,5] 5 [7,9]

# InPlace
# Stable

def Quicksort(xs : List[a], start : int = 0, end : int = len(xs) - 1):
    
    def partition(start_ : int, end_:int):
        # Hoare 
        # [3,5,9,0,1,5,9,7,2,1,3,7]
        #      ^     ^     ^
        #     pre         pos
        # [3,5,2,0,1,5,9,7,9,1,3,7]
        #      ^           ^
        #
        pivot_i : int = (start_ + end_) // 2
        pivot   : a   = xs[pivot_i]
        i : int = start_ - 1
        j = end_   + 1

        while(True):
            while (i := i+1) and (xs[i] < pivot):
                pass 
            while (j := j-1) and (xs[j] > pivot):
                    pass 
            if i >= j:
                return j
            
            xs[i], xs[j] = xs[j], xs[i]

    
    
    if start >= 0 and end >= 0 and start < end:
        p = partition(start,end)
        Quicksort(xs,start,p)
        Quicksort(xs,p+1,end)

xs = [1,3,9,0,6,5,11,2]
Quicksort(xs)
print(xs)


[0, 1, 2, 3, 5, 6, 9, 11]


# Radix Sort

Radix sort solo trabaja en los enteros, pero funciona de una manera bastante interesante, si `xs=[170, 45, 75, 90, 2, 802, 2, 66]` entonces:

* Sortea `xs` de acuerdo el digito menos significativo (LSD): `xs=[{170, 90}, {2, 802, 2}, {45, 75}, {66}]`
* Sortea `xs` de acuerdo al **segundo** LSD: `xs=[{02, 802, 02}, {45}, {66}, {170, 75}, {90}]`
* Sortea `xs` de acuerdo al **tercer**  LSD: `xs=[{002, 002, 045, 066, 075, 090}, {170}, {802}]`
* `xs=[0002, 0002, 0045, 0066, 0075, 0090, 0170, 0802}]`
* El mayor digito tenia solo 3 cifras, asi que paramos: `xs = 2,2,45,66,75,90,170,802`


In [ ]:
def concat(xss : Iterable[Iterable[Any]]) -> Iterable[Any]:
    for xs in xss:
        for x in xs:
            yield x    



def Radix(xs : List[int], digit : int = 0) -> List[int]:

    xss : List[List[int]] = [[] for _ in range(10)]
    change : bool = False
    for number in xs:
        index : int = (number // 10**digit) % 10
        xss[index].append(number)
        if index != 0:
            change = True    
    
    xs = list(concat(xss))

    if change:
        return Radix(xs,digit+1)
    else:
        return xs

xs = [1,3,10,9,0,6,5,11,2,25,20,21,]
print(Radix(xs))

[0, 1, 2, 3, 5, 6, 9, 10, 11, 20, 21, 25]


# Merge Sort

Merge sort funciona de la siguiente manera:

* Todo Arreglo vacio o con un elemento esta ordenado
* Si `l` y `r` son dos arreglos ordenados, entonces `merge(l,r)` es un arreglo ordenado
* Si `arr` es un arreglo no ordenado, entonces aplicamos mergesort a ambas mitades del arreglo y las unimos con `merge`


In [80]:

# merge([0,5,6] [1,4,7])
# = [0,1,4,5,7]
# Tienes n listas ordenas, mergearlas en una lista ordenada
# heap.push(x) 


def MergeSort(xs : List[a], start : int = 0, end : int = len(xs) -1):
    def merge(x : int, y : int, mid : int):
        left  : List[a] = [xs[i] for i in range(start,mid+1)]
        right : List[a] = [xs[i] for i in range(mid+1,end+1)]
        for i in range(start,end+1):
            if left == []:
                left = right
            if right == []:
                right = left
            
            if left[0] <= right[0]:
                xs[i] = left[0]
                left.pop(0)
            else:
                xs[i] = right[0]
                right.pop(0)

    if (start>=end):
        return
    if (start+1==end):
        xs[start],xs[end] = min(xs[start],xs[end]), max(xs[start],xs[end])
        return
    
    mid : int = (start+end) // 2
    MergeSort(xs,start,mid)
    MergeSort(xs,mid+1,end)
    merge(start,mid+1, mid)

    

xs = [2,1,3,1,2]
MergeSort(xs)
print(xs)



[1, 1, 2, 2, 3]


# Minimum Window Substring

Given two strings `s` and `t` of lengths `m` and `n` respectively, return the **minimum window substring** of `s` such that every character in `t` (including duplicates) is included in the window. If there is no such substring, return the empty string "".

The testcases will be generated such that the answer is unique.

A substring is a contiguous sequence of characters within the string.

```
Input: 
    s = "ADOBECODEBANC", 
    t = "ABC"

Output: "BANC"
Explanation: The minimum window substring "BANC" includes 'A', 'B', and 'C' from string t.
```

```
Input: 
    s = "a", 
    t = "aa"
Output: ""
Explanation: Both 'a's from t must be included in the window.
Since the largest window of s only has one 'a', return empty string.

```


In [ ]:
#BCA

# Array List?
# Dynamic Array?

'''

0 3 5 9 10 12 
A B C B A   C

sol_min = inf
{ABC}
         0 A {(A,0)}               | {A:0, B:1, C:1}
0        3 B {(A,0),(B,3)}         | {A:0, B:0, C:1}
3        5 C {(A,0),(B,2),(C,5)}   | {A:0, B:0, C:0} -> solMin: min(|0-5|,inf) = 5
5 }      9 B {(B,2),(C,5),(B,9)}   | {A:1,B:-1, C:0} -> Se puede modificar para que popee el B mas antiguo
        10 A {(C,5),(B,9),(A,10)}  | {A:0, B:0, C:0} -> solMin: min(|5-10|,5) = 5
        12 C {(B,9),(A,10),(C,12)} | {A:0, B:0, C:0} -> solMin: min(|9-12|,5) = 3
'''

# Regular Expression Matcher

Dada una expresion regular `p` compuesta de los siguientes patrones: 

* `x*` matchea 0 o mas veces el patron `x`
* `.`  matchea cualquier caracter 1 sola vez

Determine si un string `s` matchea `p`


In [81]:
# aaav
# a*av

#  avavde
# (avd)*jds(va)*

#   /\         /\
#   \/         \/
#   av ->  -> av



# 
# DFA: Deterministic Finite Automaton # DFS
# NFA: Non-Deterministic Finite Automaton #BFS
#  ------           ------
#  |    |           |    |
#  v    |           v    |
# (a -> v) -> . -> (a -> v)

# NFA

# Making File Names Unique

Given an array of strings `names` of size `n`. You will create `n` folders in your file system such that, at the `i`th minute, you will create a folder with the name `names[i]`.

Since two files cannot have the same name, if you enter a folder name which is previously used, the system will have a suffix addition to its name in the form of `(k)`, where, `k` is the smallest positive integer such that the obtained name remains unique.

Return an array of strings of length n where ans[i] is the actual name the system will assign to the ith folder when you create it.

```
Input: 
    names = ["gta","gta(1)","gta","avalon"]
Output: 
    ["gta","gta(1)","gta(2)","avalon"]
Explanation: Let's see how the file system creates folder names:
"gta" --> not assigned before, remains "gta"
"gta(1)" --> not assigned before, remains "gta(1)"
"gta" --> the name is reserved, system adds (k), since "gta(1)" is also reserved, systems put k = 2. it becomes "gta(2)"
"avalon" --> not assigned before, remains "avalon"
```


```
Input: 
    names = ["kaido","kaido(1)","kaido","kaido(1)"]
Output: 
    ["kaido","kaido(1)","kaido(2)","kaido(1)(1)"]
Explanation: Please note that system adds the suffix (k) to current name even it contained the same suffix before.

```

In [ ]:
#sol

# g
# t
# a <-> v -> l -> o -> n
# ( 
# 1
# )   


# Valid Ip Addresses

Given a string containing only digits, restore it by returning all possible valid IP address combinations.

A valid IP address must be in the form of A.B.C.D, where A,B,C and D are numbers from 0-255. The numbers cannot be 0 prefixed unless they are 0.

```
Input: 
    s = "25525511135"
Output:
     ["255.255.11.135", "255.255.111.35"]

```

In [ ]:
#0-255.0-255.0-255.0-255
# 25.52.55.111
# 192.168.1.1
# 8.8.8.8

# 12016
# 12.0.1.6
# 1.20.1.6

# 12016 -> 1.
#       -> 1

# NFA
# [0-255].[0-255].[0-255].[0-255]